In [2]:
from pypeg2 import *


class Type(Keyword):
    grammar = Enum( K("int"), K("long") )
    
class Parameter(object):
    grammar = attr("typing", Type), blank, name()
    
class Parameters(Namespace):
    grammar = optional(csl(Parameter))
    

class Instruction(str):
    def heading(self, parser):
        return "/* on level " + str(parser.indention_level) + " */", endl

    grammar = heading, word, ";", endl
    
block = "{", endl, maybe_some(indent(Instruction)), "}", endl

class Function(List):
    grammar = attr("typing", Type), blank, name(), "(", attr("parms", Parameters), ")", endl, block


In [3]:
f = parse("int f(int a, long b) { do_this; do_that; }", Function)

In [4]:
f.typing

Type('int')

In [5]:
from pypeg2 import *

class Method(Keyword):
    grammar = Enum(K("GET"), K("HEAD"), K("POST"), K("PUT"), K("DELETE"), K("TRACE"), K("CONNECT"), K("OPTIONS"))

class RequestURI(str):
    grammar = re.compile(r"\/\w+\.\w+")

class ReqLine:
    grammar = attr("method", Method), blank, attr("request_uri", RequestURI)


In [6]:
f = parse("GET /hello.htm", ReqLine)

In [7]:
f.method

Method('GET')

In [16]:
class Method(Keyword):
    grammar = Enum(K("GET"), K("HEAD"), K("POST"), K("PUT"), K("DELETE"), K("TRACE"), K("CONNECT"), K("OPTIONS"))

class Digit:
    grammar = re.compile(r"[0-9]")

class SubDelimiter(Keyword):
    grammar = Enum(K("!"), K("$"), K("&"), K("'"), K("("), K(")"), K("*"), K("+"), K(","), K(";"), K("="))

# Alpha | Digit 
#   <=> [a-zA-Z0-9_] 
#   <=> \w
class Unreserved:
    grammar = re.compile(r"\w"), "-", ".", "_", "~"

# Pour host, on ignore pct-encoded
class RegularName:
    grammar = maybe_some([Unreserved, SubDelimiter]) 

#Je decide de skip pct-encoded pour l'instant
class PChar:
    grammar = [Unreserved, SubDelimiter, ":", "@"]

#Start Authority
class Fragment:
    grammar = maybe_some([PChar, "/", "?"])

class DecOctet:
    grammar = [Digit, ([1-9], Digit), ("1", Digit, Digit), ("2", [0-4], Digit), ("25", [0-5])]

# Not considering IPv6 adresses for the moment
class IPAddr:
    grammar = DecOctet, ".", DecOctet, ".", DecOctet, ".", DecOctet

#test syntax ?
class Host:
    grammar = attr("identifier", [IPAddr, RegularName])

class Port:
    grammar = maybe_some(Digit)

class Auth:
    grammar = Host, maybe_some(":", Port)
# end Authority

#start Path
class Segment:
    grammar = maybe_some(PChar)
class Path:
    grammar = Segment, maybe_some("/", Segment)
#end Path

class HierPart:
    grammar = "//", Auth, Path


#TODO: changer cette expreg (marche pas)
class RequestURI:
    grammar = re.compile(r"^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?")

class HttpVer:
    grammar = "HTTP", "/", some(Digit), ".", some(Digit)

class ReqLine:
    grammar = attr("method", Method), whitespace, attr("request_uri", RequestURI), whitespace, attr("http_version", HttpVer), endl

#DEBUG
class Header:
    grammar = re.compile(r"header")

class MessageBody:
    grammar = re.compile(r"body")

class Request:
    grammar = attr("req_line", ReqLine), maybe_some(attr("header", Header), endl), endl, optional(attr("mess_body", MessageBody))


In [17]:
request = """GET /hello.htm
header
mess_body """

f = parse(request, ReqLine)
f.method

SyntaxError: expecting match on (?m)\s+ (<string>)